In [1]:
import os
import json
import numpy as np
import io
from datetime import datetime
from data_generator import DataGenerator
from trade_env import TraderEnv
from rnn.model_keras import NeuralNetwork
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from sklearn.preprocessing import MinMaxScaler

/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dt = DataGenerator(random=True, first_index=10)
dt.rewind()

In [3]:
# integer encode input data
def onehot_encoded (integer_encoded, char_to_int = 3):
    # one hot encode
    onehot_encoded = list()
    letter = [0 for _ in range(char_to_int)]
    letter[integer_encoded] = 1
    onehot_encoded.append(letter)
    
    return onehot_encoded[0]

In [4]:
last_price = 0
steps = 100
def get_state(raw_state):
    global last_price
    list = []

    price = raw_state["price"]

    def prepare_orders(orders, price, multi):
        for order in orders:
            list.append((float(order[0])/price) * multi)
            #list.append(float(order[1]))

    bids = raw_state["bids"][:5]
    asks = raw_state["asks"][:5]
    prepare_orders(bids, price, 1)
    prepare_orders(asks, price, -1)


    if last_price != 0:
        list.extend([price/last_price])
    else:
        list.extend([0])
        
    last_price = price
    
    y = dt.get_from_index(dt.index + steps)["price"]
    
    if y > price:
        y = onehot_encoded(0)
    elif y < price: 
        y = onehot_encoded(2)
    else:
        y = onehot_encoded(1)
        
    #datetime.fromtimestamp(int(state['timestamp']))
    
    return [list, y]


In [5]:
trainX = []
trainY = []
while len(trainX) < 80000: #(dt.max_steps() - steps*3):
    state = dt.next()
    state = get_state(state)
    trainX.append(state[0])
    trainY.append(state[1])
trainX = np.array(trainX)
trainY = np.array(trainY)
print(trainX.shape)
print(trainY.shape)

(80000, 11)
(80000, 3)


In [6]:
def get_batches(data, features, batch_size, seq_length):
    int_text = np.array(data)
    
    n_batches = int(len(data) / (batch_size * seq_length))

    xdata = np.array(data[: n_batches * batch_size * seq_length])
    
    data = xdata.reshape(-1, seq_length, features)
    
    #step1 = xdata.reshape(batch_size, -1)
    
    #print (step1.shape)
    #data = np.split(step1, n_batches, 1)
    
    return np.array(list(data))

In [7]:
print(trainX.shape)
batch_size = 32
seq_length = 50
xdata = get_batches(trainX, trainX.shape[1], batch_size, seq_length)
print(xdata.shape)

(80000, 11)
(1600, 50, 11)


In [8]:
print(trainY.shape)
ydata = get_batches(trainY, trainY.shape[1], batch_size, seq_length)
#ydata = ydata.reshape(9900, seq_length, 3)
print(ydata.shape)

(80000, 3)
(1600, 50, 3)


In [9]:
y_train_reshaped = []
for i in range(ydata.shape[0]):
    y_train_reshaped.append(ydata[i, -1])

In [10]:
y_train_reshaped = np.array(y_train_reshaped)

In [ ]:
nn = NeuralNetwork(0.00001, xdata.shape[2], y_train_reshaped.shape[1], xdata.shape[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 50, 11)            759       
_________________________________________________________________
gru_2 (GRU)                  (None, 11)                759       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1536      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
__________

In [ ]:
nn.train(xdata, y_train_reshaped)

Train on 1120 samples, validate on 480 samples
Epoch 1/150
1120/1120 [==============================] - 4s 3ms/step - loss: 0.9064 - mean_absolute_error: 0.3858 - acc: 0.4982 - val_loss: 0.7270 - val_mean_absolute_error: 0.3416 - val_acc: 0.5333
Epoch 2/150
1120/1120 [==============================] - 2s 2ms/step - loss: 0.7714 - mean_absolute_error: 0.3419 - acc: 0.5250 - val_loss: 0.7594 - val_mean_absolute_error: 0.3324 - val_acc: 0.5333
Epoch 3/150
1120/1120 [==============================] - 2s 2ms/step - loss: 0.7779 - mean_absolute_error: 0.3427 - acc: 0.5375 - val_loss: 0.7143 - val_mean_absolute_error: 0.3385 - val_acc: 0.4646
Epoch 4/150
1120/1120 [==============================] - 2s 2ms/step - loss: 0.7747 - mean_absolute_error: 0.3434 - acc: 0.5286 - val_loss: 0.7192 - val_mean_absolute_error: 0.3322 - val_acc: 0.5333
Epoch 5/150
1120/1120 [==============================] - 2s 2ms/step - loss: 0.7740 - mean_absolute_error: 0.3426 - acc: 0.5027 - val_loss: 0.7213 - val_mean

In [ ]:
dt.index